<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-09 14:39:07
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: 2.05 L (1.48%)
Current PnL: -28.80 L (-18.37%)
CY Booked + Current PnL: -14.06 L (-8.97%)
-------------------
Total profit:  1.42 L
Total loss:  -30.23 L
-------------------
Total Booked + Current PnL: 12.92 L (9.95%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.48%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.19 L (66.95%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.14,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,60.0,H-SC,3.46,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
18,COALINDIA,-0.55,5.84,12.65,19.23,20145.0,8788.0,159248.0,484.83,24.27,54.0,L-LC,12.62,182.0,0.44,1.13,28.36,XY25,ATH,MINING
77,TTKPRESTIG,1.52,-24.94,33.36,0.09,25233.0,-25138.0,75639.0,770.00,75.67,45.0,M-SC,8.44,253.0,-1.00,0.54,4.10,OX40N,NTT,DURABLES
51,MEDANTA,1.57,-3.09,28.04,24.08,34496.0,-3926.0,123024.0,1486.00,-10.94,56.0,X-SC,11.33,91.0,-0.11,0.87,12.68,XY24,NTT,HEALTHCARE
36,ICICIGI,1.30,-0.11,19.47,19.34,38920.0,-214.0,199895.0,2252.93,-21.21,53.0,X-MC,3.61,71.0,-0.01,1.42,15.81,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,15.80,-18.89,98.20,60.77,145584.0,-34522.0,148253.0,521.0,70.80,66.0,H-SC,3.61,158.0,-0.24,1.05,34.41,XR,NTT,JEWELLERY
7,ATULAUTO,10.00,-15.42,72.10,45.57,120217.0,-30396.0,166736.0,844.0,3613.51,72.0,M-SC,8.69,245.0,-0.25,1.19,26.18,XY24,NTT,AUTO
19,COFFEEDAY,6.14,-45.91,150.08,35.27,92181.0,-52128.0,61421.0,80.0,-57.75,48.0,L-SC,13.77,268.0,-0.57,0.44,50.33,XR,NTT,HOTELS
83,WHIRLPOOL,5.29,-27.89,152.26,81.92,126050.0,-32012.0,82786.0,2270.0,-57.77,66.0,M-SC,32.62,218.0,-0.25,0.59,16.29,XR,NTT,DURABLES
31,HATHWAY,4.90,-35.04,166.78,73.29,117780.0,-38100.0,70620.0,31.4,718.84,55.0,H-SC,19.66,172.0,-0.32,0.50,15.17,XR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,-2.27,-16.60,50.50,25.51,78376.0,-30895.0,155200.0,4671.50,-4.77,51.0,H-LC,3.47,50.0,-0.39,1.10,14.37,AR,ATH,ELECTRICAL
52,PAGEIND,-2.00,-11.79,48.08,30.62,100535.0,-27954.0,209100.0,51605.07,-37.34,53.0,X-MC,19.76,57.0,-0.28,1.49,7.16,X40,ATH,APPARELS
57,RECLTD,-1.32,0.84,19.73,20.74,40422.0,1705.0,204875.0,446.00,33.49,56.0,M-MC,4.00,189.0,0.04,1.46,12.90,XY25,NTT,FINANCE
44,ITC,-0.86,-20.84,39.94,10.78,75985.0,-50071.0,190247.0,452.00,-53.00,44.0,X-LC,18.80,1.0,-0.66,1.35,4.79,X40,NTT,FMCG
13,BERGEPAINT,-0.84,-16.55,45.16,21.14,85678.0,-37617.0,189722.0,680.00,-22.51,30.0,X-MC,9.15,75.0,-0.44,1.35,1.20,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.82,0.43,107.02,107.91,158691.0,638.0,148282.0,1641.55,-18.12,64.0,M-SC,11.3,220.0,0.00,1.05,11.56,OX40N,ATH,CABLES
57,RECLTD,-1.32,0.84,19.73,20.74,40422.0,1705.0,204875.0,446.00,33.49,56.0,M-MC,4.0,189.0,0.04,1.46,12.90,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.82,0.43,107.02,107.91,158691.0,638.0,148282.0,1641.55,-18.12,64.0,M-SC,11.30,220.0,0.00,1.05,11.56,OX40N,ATH,CABLES
66,SIS,0.81,-21.03,53.56,21.26,46962.0,-23350.0,87682.0,528.00,2072.54,65.0,H-SC,4.54,163.0,-0.50,0.62,20.88,OX40N,NTT,MISC
72,TATAELXSI,0.64,-24.60,75.10,32.03,74655.0,-32427.0,99408.0,9161.00,-18.39,42.0,H-SC,6.33,157.0,-0.43,0.71,11.35,OX40N,NTT,IT
77,TTKPRESTIG,1.52,-24.94,33.36,0.09,25233.0,-25138.0,75639.0,770.00,75.67,45.0,M-SC,8.44,253.0,-1.00,0.54,4.10,OX40N,NTT,DURABLES
40,INDIGOPNTS,3.28,-25.98,35.12,0.02,45378.0,-45351.0,129208.0,1408.00,96.99,38.0,M-SC,5.83,240.0,-1.00,0.92,12.39,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-1.32,0.84,19.73,20.74,40422.0,1705.0,204875.0,446.00,33.49,56.0,M-MC,4.00,189.0,0.04,1.46,12.90,XY25,NTT,FINANCE
18,COALINDIA,-0.55,5.84,12.65,19.23,20145.0,8788.0,159248.0,484.83,24.27,54.0,L-LC,12.62,182.0,0.44,1.13,28.36,XY25,ATH,MINING
50,MASFIN,2.14,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,60.0,H-SC,3.46,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
26,FINCABLES,2.82,0.43,107.02,107.91,158691.0,638.0,148282.0,1641.55,-18.12,64.0,M-SC,11.30,220.0,0.00,1.05,11.56,OX40N,ATH,CABLES
1,ABB,-0.15,6.55,36.62,45.57,102082.0,17141.0,278760.0,7934.00,-31.49,72.0,H-MC,3.84,121.0,0.17,1.98,23.79,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.15,6.55,36.62,45.57,102082.0,17141.0,278760.0,7934.00,-31.49,72.0,H-MC,3.84,121.0,0.17,1.98,23.79,AR,NTT,ELECTRICAL
50,MASFIN,2.14,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,60.0,H-SC,3.46,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
26,FINCABLES,2.82,0.43,107.02,107.91,158691.0,638.0,148282.0,1641.55,-18.12,64.0,M-SC,11.30,220.0,0.00,1.05,11.56,OX40N,ATH,CABLES
39,INDIAMART,0.61,-3.23,113.62,106.72,135612.0,-3980.0,119356.0,4810.62,-52.15,54.0,H-SC,8.10,138.0,-0.03,0.85,19.40,AR,ATH,MISC
84,WIPRO,-0.16,-5.52,82.33,72.27,131805.0,-9355.0,160093.0,420.00,-18.78,36.0,M-LC,3.56,101.0,-0.07,1.14,2.90,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,0.46,-15.69,43.45,20.95,92229.0,-39503.0,212265.0,3460.25,-16.51,29.0,X-LC,8.78,36.0,-0.43,1.51,14.35,X40,ATH,PAINTS
13,BERGEPAINT,-0.84,-16.55,45.16,21.14,85678.0,-37617.0,189722.0,680.00,-22.51,30.0,X-MC,9.15,75.0,-0.44,1.35,1.20,XY24,NTT,PAINTS
42,INFY,-0.73,1.85,31.81,34.24,101369.0,5781.0,318669.0,1972.00,-20.67,30.0,X-LC,5.70,6.0,0.06,2.27,10.28,X40,NTT,IT
73,TCS,-0.06,-19.16,46.66,18.55,142663.0,-72487.0,305750.0,4311.59,-30.42,34.0,X-LC,0.95,3.0,-0.51,2.17,4.02,X40,ATH,IT
8,AWL,-0.13,-31.59,129.77,57.19,289532.0,-103015.0,223112.0,485.00,-67.71,35.0,X-MC,16.80,58.0,-0.36,1.59,1.68,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-0.06,-19.16,46.66,18.55,142663.0,-72487.0,305750.0,4311.59,-30.42,34.0,X-LC,0.95,3.0,-0.51,2.17,4.02,X40,ATH,IT
81,VBL,4.26,-7.53,46.73,35.67,136472.0,-23798.0,292044.0,671.64,-18.58,43.0,X-LC,0.98,5.0,-0.17,2.08,5.28,X40N,ATH,FMCG
5,ANGELONE,2.25,10.63,12.27,24.20,41434.0,32438.0,337688.0,3033.00,57.42,59.0,X-SC,1.62,99.0,0.78,2.40,41.43,X40N,NTT,FINANCE
21,DABUR,0.61,-0.02,43.72,43.70,108438.0,-39.0,248029.0,735.00,-8.33,52.0,X-MC,1.97,73.0,-0.00,1.76,15.03,XY24,BTT,FMCG
34,HINDUNILVR,0.34,-5.37,20.12,13.67,48942.0,-13813.0,243250.0,2922.00,-12.44,60.0,X-LC,2.42,9.0,-0.28,1.73,14.84,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.84,-16.55,45.16,21.14,85678.0,-37617.0,189722.0,680.00,-22.51,30.0,X-MC,9.15,75.0,-0.44,1.35,1.20,XY24,NTT,PAINTS
8,AWL,-0.13,-31.59,129.77,57.19,289532.0,-103015.0,223112.0,485.00,-67.71,35.0,X-MC,16.80,58.0,-0.36,1.59,1.68,XY24,NTT,FMCG
2,ABBOTINDIA,0.40,-7.83,28.10,18.07,46323.0,-14005.0,164850.0,35195.00,-21.05,47.0,X-MC,9.48,64.0,-0.30,1.17,3.15,X40,ATH,PHARMA
53,PGHH,0.27,-10.62,47.69,32.01,91015.0,-22672.0,190848.0,17616.87,-37.21,45.0,X-MC,8.84,60.0,-0.25,1.36,3.16,X40,ATH,FMCG
9,BAJAJHFL,-0.01,-21.82,98.86,55.47,188039.0,-53079.0,190207.0,181.50,-30.61,46.0,X-MC,17.30,63.0,-0.28,1.35,3.69,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.79,-28.75,100.47,42.84,46536.0,-18688.0,46318.0,424.00,-53.17,58.0,X-SC,20.84,83.0,-0.40,0.33,11.42,XY24,NTT,MISC
58,RELAXO,0.99,-51.71,215.37,52.31,150985.0,-75055.0,70105.0,1176.00,-48.60,43.0,X-SC,12.80,92.0,-0.50,0.50,4.25,X40N,NTT,FOOTWEAR
12,BATAINDIA,3.93,-43.07,137.51,35.20,100723.0,-55422.0,73248.0,2096.00,-3.97,47.0,X-SC,21.24,93.0,-0.55,0.52,4.09,X40,NTT,FOOTWEAR
51,MEDANTA,1.57,-3.09,28.04,24.08,34496.0,-3926.0,123024.0,1486.00,-10.94,56.0,X-SC,11.33,91.0,-0.11,0.87,12.68,XY24,NTT,HEALTHCARE
35,HONAUT,1.63,-20.24,79.70,43.32,103530.0,-32972.0,129900.0,58357.33,-31.51,46.0,X-SC,9.60,90.0,-0.32,0.92,5.47,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.86,-20.84,39.94,10.78,75985.0,-50071.0,190247.0,452.00,-53.00,44.0,X-LC,18.80,1.0,-0.66,1.35,4.79,X40,NTT,FMCG
75,TMPV,2.10,-22.68,59.05,22.98,103141.0,-51231.0,174667.0,600.00,-19.41,64.0,X-LC,4.00,2.0,-0.50,1.24,11.66,XY24,NTT,AUTO
73,TCS,-0.06,-19.16,46.66,18.55,142663.0,-72487.0,305750.0,4311.59,-30.42,34.0,X-LC,0.95,3.0,-0.51,2.17,4.02,X40,ATH,IT
81,VBL,4.26,-7.53,46.73,35.67,136472.0,-23798.0,292044.0,671.64,-18.58,43.0,X-LC,0.98,5.0,-0.17,2.08,5.28,X40N,ATH,FMCG
42,INFY,-0.73,1.85,31.81,34.24,101369.0,5781.0,318669.0,1972.00,-20.67,30.0,X-LC,5.70,6.0,0.06,2.27,10.28,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,2.34,-31.73,94.66,32.89,49893.0,-24498.0,52708.0,1800.00,-750.65,59.0,L-MC,11.47,259.0,-0.49,0.37,45.15,XR,NTT,BANKS
67,SONACOMS,3.06,-8.45,51.93,39.09,48092.0,-8549.0,92610.0,804.02,-28.36,67.0,M-SC,1.73,227.0,-0.18,0.66,31.01,AR,ATH,AUTO
50,MASFIN,2.14,1.47,19.81,21.57,19695.0,1440.0,99420.0,397.04,-12.38,60.0,H-SC,3.46,168.0,0.07,0.71,45.08,XR,ATH,FINANCE
14,BSOFT,1.90,-19.50,87.32,50.79,98864.0,-27433.0,113220.0,831.70,3.21,56.0,H-SC,7.14,171.0,-0.28,0.80,31.83,XR,ATH,IT
79,VAIBHAVGBL,15.80,-18.89,98.20,60.77,145584.0,-34522.0,148253.0,521.00,70.80,66.0,H-SC,3.61,158.0,-0.24,1.05,34.41,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,15.80,-18.89,98.20,60.77,145584.0,-34522.0,148253.0,521.00,70.80,66.0,H-SC,3.61,158.0,-0.24,1.05,34.41,XR,NTT,JEWELLERY
5,ANGELONE,2.25,10.63,12.27,24.20,41434.0,32438.0,337688.0,3033.00,57.42,59.0,X-SC,1.62,99.0,0.78,2.40,41.43,X40N,NTT,FINANCE
67,SONACOMS,3.06,-8.45,51.93,39.09,48092.0,-8549.0,92610.0,804.02,-28.36,67.0,M-SC,1.73,227.0,-0.18,0.66,31.01,AR,ATH,AUTO
1,ABB,-0.15,6.55,36.62,45.57,102082.0,17141.0,278760.0,7934.00,-31.49,72.0,H-MC,3.84,121.0,0.17,1.98,23.79,AR,NTT,ELECTRICAL
7,ATULAUTO,10.00,-15.42,72.10,45.57,120217.0,-30396.0,166736.0,844.00,3613.51,72.0,M-SC,8.69,245.0,-0.25,1.19,26.18,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.22
1,25,44.44
2,50,76.40


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.67
MC    30.82
LC    23.51
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.54
X40      23.24
X40N     15.15
AR        8.97
XR        8.94
XY25      8.79
OX40N     7.47
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.11
H-SC    22.96
X-LC    20.14
M-SC    11.59
X-SC    10.34
H-MC     4.88
M-MC     1.46
M-LC     1.14
L-LC     1.13
H-LC     1.10
L-SC     0.78
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.93,-12.80,50.70
FINANCE,13.49,-10.17,53.53
IT,12.18,-24.87,87.25
MISC,7.13,-29.39,82.33
ELECTRICAL,6.05,-10.12,50.30
PAINTS,5.14,-27.71,46.68
INSURANCE,4.71,-2.36,37.39
PHARMA,4.05,-3.17,35.64
AUTO,3.09,-20.78,62.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237035.0,21
AR,1309999.0,10
XR,1231598.0,12
X40,1229939.0,15
X40N,991022.0,10
OX40N,741439.0,10
XY25,384037.0,6
SR,294051.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3561290.0,24
X-MC,1638009.0,16
M-SC,1430660.0,15
X-LC,1059414.0,12
X-SC,835515.0,9
H-MC,392104.0,3
L-SC,181487.0,2
M-LC,131805.0,1
H-LC,78376.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233176.0      6
           AR         928826.0      5
M-SC       XY24       805330.0      6
H-SC       XR         782047.0      7
X-MC       X40        543975.0      7
X-LC       X40        517753.0      6
X-MC       XY24       483648.0      3
           X40N       399101.0      4
X-SC       X40N       366559.0      4
M-SC       OX40N      328943.0      5
H-SC       OX40N      323190.0      4
X-SC       XY24       300745.0      3
H-SC       SR         294051.0      2
X-LC       X40N       225362.0      2
X-MC       XY25       211285.0      2
H-MC       XY24       210022.0      1
X-LC       XY24       204114.0      2
H-MC       AR         182082.0      2
M-SC       XR         175672.0      2
X-SC       X40        168211.0      2
M-LC       XR         131805.0      1
M-SC       AR         120715.0      2
X-LC       XY25       112185.0      2
L-SC       XR          92181.0      1
           OX40N       89306.0      1
H-LC       AR          78376.0      1
L-MC       XR          49893.0      1
M-MC       XY25        40422.0      1
L-LC       XY25        20145.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
